# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的车辆销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [car_sales_data.txt](car_sales_data.txt) 文件中

In [1]:
with open("car_sales_data.txt", "r", encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 130,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 我对这款车的燃油效率很关心，能告诉我吗？\n   [销售回答] 这款车型在城市道路上的燃油经济性非常出色，每加仑能行驶约30英里，高速路上更能达到40英里左右。')

In [6]:
len(docs)

90

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url="https://api.xiaoai.plus/v1", api_key="sk-WUVjTMqzcNGj4D4d6dC5Ce1520E841C59a787011D9CeAb8c"))

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "小区吵不吵"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我对车辆的噪音水平比较敏感，这款车在高速行驶时会有噪音吗？
    [销售回答] 车辆经过了有效的隔音设计和噪音控制技术，高速行驶时的内部噪音非常低，让您的驾驶体验更加静谧舒适。

[客户问题] 我对车辆的静音设计很关注，这款车的噪音控制如何？
   [销售回答] 这款车辆经过了精心设计和优化，采用了有效的隔音材料和噪音控制技术，能够有效减少路面噪音和风噪，为驾驶员和乘客提供安静舒适的驾乘体验。

[客户问题] 这款车的驾驶感受如何？
    [销售回答] 驾驶感受非常平稳和舒适，悬挂系统设计使得车辆在各种路况下表现出色，无论是城市道路还是高速公路。

[客户问题] 我们家有几个小孩，这款车适合家庭使用吗？
    [销售回答] 这款车是家庭首选，拥有宽敞的内部空间和多项安全功能，能够满足您日常的家庭用车需求。



In [11]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002294852A680>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 我对车辆的噪音水平比较敏感，这款车在高速行驶时会有噪音吗？
    [销售回答] 车辆经过了有效的隔音设计和噪音控制技术，高速行驶时的内部噪音非常低，让您的驾驶体验更加静谧舒适。

[客户问题] 我对车辆的静音设计很关注，这款车的噪音控制如何？
   [销售回答] 这款车辆经过了精心设计和优化，采用了有效的隔音材料和噪音控制技术，能够有效减少路面噪音和风噪，为驾驶员和乘客提供安静舒适的驾乘体验。

[客户问题] 这款车的驾驶感受如何？
    [销售回答] 驾驶感受非常平稳和舒适，悬挂系统设计使得车辆在各种路况下表现出色，无论是城市道路还是高速公路。



In [15]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的车啊？")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我还在犹豫不决，有没有可供试驾的车辆？
   [销售回答] 当然可以安排试驾，让您亲自感受车辆的驾驶体验。请告诉我们您方便的时间，我们会为您预定试驾车辆。

[客户问题] 我们家有几个小孩，这款车适合家庭使用吗？
    [销售回答] 这款车是家庭首选，拥有宽敞的内部空间和多项安全功能，能够满足您日常的家庭用车需求。

[客户问题] 我需要贷款购车，你们有推荐的金融服务吗？
   [销售回答] 我们和多家信誉良好的金融服务合作，可以为您提供多种贷款选择，帮助您找到最适合的贷款方案和最低利率。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.6}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我对车辆的噪音水平比较敏感，这款车在高速行驶时会有噪音吗？
    [销售回答] 车辆经过了有效的隔音设计和噪音控制技术，高速行驶时的内部噪音非常低，让您的驾驶体验更加静谧舒适。

[客户问题] 我对车辆的静音设计很关注，这款车的噪音控制如何？
   [销售回答] 这款车辆经过了精心设计和优化，采用了有效的隔音材料和噪音控制技术，能够有效减少路面噪音和风噪，为驾驶员和乘客提供安静舒适的驾乘体验。

[客户问题] 这款车的驾驶感受如何？
    [销售回答] 驾驶感受非常平稳和舒适，悬挂系统设计使得车辆在各种路况下表现出色，无论是城市道路还是高速公路。

[客户问题] 我们家有几个小孩，这款车适合家庭使用吗？
    [销售回答] 这款车是家庭首选，拥有宽敞的内部空间和多项安全功能，能够满足您日常的家庭用车需求。



### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题] 我对车辆的噪音水平比较敏感，这款车在高速行驶时会有噪音吗？\n    [销售回答] 车辆经过了有效的隔音设计和噪音控制技术，高速行驶时的内部噪音非常低，让您的驾驶体验更加静谧舒适。'

In [21]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我对车辆的噪音水平比较敏感，这款车在高速行驶时会有噪音吗？\n    ',
 '车辆经过了有效的隔音设计和噪音控制技术，高速行驶时的内部噪音非常低，让您的驾驶体验更加静谧舒适。']

In [22]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [23]:
ans

'车辆经过了有效的隔音设计和噪音控制技术，高速行驶时的内部噪音非常低，让您的驾驶体验更加静谧舒适。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [25]:
query = "我想油耗少一点"

print(sales(query))

[]


C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [26]:
print(sales(query, 0.75))

['油耗通常受到行驶条件、驾驶风格和车辆维护状况的影响，但我们的技术团队会确保实现官方公布的燃油经济性标准。', '这款车型在城市道路上的燃油经济性非常出色，每加仑能行驶约30英里，高速路上更能达到40英里左右。', '车辆的具体油耗数据会根据车型和驾驶条件而有所不同。我们可以为您提供详细的城市和高速路段的油耗数据，并根据您的驾驶习惯提供节能驾驶建议。']


In [27]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们可以根据您的具体购车需求提供最优惠的价格和多种购车优惠活动，包括现金折扣、贷款优惠和车辆交换计划等。', '这款车提供三年或36,000英里的整车保修，同时还包括五年或60,000英里的动力总成保修，让您购车后更加放心。', '这款车型在城市道路上的燃油经济性非常出色，每加仑能行驶约30英里，高速路上更能达到40英里左右。', '这款车经过严格的质量控制和测试，拥有优秀的可靠性记录。此外，它还提供了长达10年或100,000英里的动力总成保修，让您更加放心。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-WUVjTMqzcNGj4D4d6dC5Ce1520E841C59a787011D9CeAb8c", model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [29]:
qa_chain({"query": "你们有200万的车吗？"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们有200万的车吗？',
 'result': '对不起，我没有关于车辆销售的信息。我建议你联系当地的车辆经销商或者查看一些在线汽车销售平台来获取更多信息。'}

In [34]:
qa_chain({"query": "汽车吵不吵"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '汽车吵不吵',
 'result': '这个问题的答案可能因汽车的类型和品牌而异。一般来说，内燃机汽车在运行时会比电动汽车产生更多的噪音。如果您具体指某一款汽车，我可能需要更多的信息才能给出准确回答。'}

In [35]:
print(sales("汽车声音大不大"))

[]


C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(base_url="https://api.xiaoai.plus/v1", api_key="sk-WUVjTMqzcNGj4D4d6dC5Ce1520E841C59a787011D9CeAb8c"), allow_dangerous_deserialization=True)

In [20]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-WUVjTMqzcNGj4D4d6dC5Ce1520E841C59a787011D9CeAb8c", model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [21]:
qa_chain({"query": "我想买豪车，你们有么"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买豪车，你们有么', 'result': '对不起，我不能帮助你购买豪车。我是一个人工智能，专门用来提供信息和答疑解惑的。'}

In [13]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [15]:
qa_chain({"query": "我想买豪车，你们有么"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买豪车，你们有么',
 'result': '对不起，我无法提供购买豪车的服务。我是一个人工智能助手，我可以提供信息和回答问题，但我不能卖东西。'}

In [16]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [17]:
result = qa_chain({"query": "我想买豪车，你们有么"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [18]:
result

{'query': '我想买豪车，你们有么',
 'result': '对不起，我不能帮助你购买豪车。我是一个人工智能，我主要用来提供信息和回答问题。',
 'source_documents': []}